<a href="https://colab.research.google.com/github/waqqasansari/lets_Play_with_PyTorch/blob/master/10_2_pytorch_CNN_INCEPTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [18]:
batch_size = 64

#Download MNIST dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                               train=False,
                               transform=transforms.ToTensor())
                               

#Data Loader (Input PipeLine)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

class InceptionA(nn.Module):
  def __init__(self, in_channels):
    super(InceptionA, self).__init__()
    self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size = 1)

    self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
    self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

    self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
    self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

    self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)


  def forward(self, x):
    branch1x1 = self.branch1x1(x)

    branch5x5 = self.branch5x5_1(x)
    branch5x5 = self.branch5x5_2(branch5x5)

    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

    branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
    branch_pool = self.branch_pool(branch_pool)

    outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
    return torch.cat(outputs, 1)

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

    self.incept1 = InceptionA(in_channels=10)
    self.incept2 = InceptionA(in_channels=20)

    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(1408, 10)


  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = self.incept1(x)
    x = F.relu(self.mp(self.conv2(x)))
    x = self.incept2(x)
    x = x.view(in_size, -1)
    x = self.fc(x)
    return F.log_softmax(x)

model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),
                                                                   100. * batch_idx / len(train_loader), loss.item()))  
def test():
  model.eval()
  test_loss = 0
  correct = 0

  for data, target in test_loader:
    data, target = Variable(data), Variable(target)
    output = model(data)
    # sum up batch loss
    test_loss += F.nll_loss(output, target, size_average=False).data
    # get the index of the max log-probability
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
  
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305184
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.303740
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.313249
Train Epoch: 1 [192/60000 (0%)]	Loss: 2.305468
Train Epoch: 1 [256/60000 (0%)]	Loss: 2.304955
Train Epoch: 1 [320/60000 (1%)]	Loss: 2.306435
Train Epoch: 1 [384/60000 (1%)]	Loss: 2.296177
Train Epoch: 1 [448/60000 (1%)]	Loss: 2.300576
Train Epoch: 1 [512/60000 (1%)]	Loss: 2.301956
Train Epoch: 1 [576/60000 (1%)]	Loss: 2.315831
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.303978
Train Epoch: 1 [704/60000 (1%)]	Loss: 2.295524
Train Epoch: 1 [768/60000 (1%)]	Loss: 2.294747
Train Epoch: 1 [832/60000 (1%)]	Loss: 2.310716
Train Epoch: 1 [896/60000 (1%)]	Loss: 2.302552
Train Epoch: 1 [960/60000 (2%)]	Loss: 2.302050
Train Epoch: 1 [1024/60000 (2%)]	Loss: 2.300989
Train Epoch: 1 [1088/60000 (2%)]	Loss: 2.293340
Train Epoch: 1 [1152/60000 (2%)]	Loss: 2.294088
Train Epoch: 1 [1216/60000 (2%)]	Loss: 2.299330
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296162
Train Epoch

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Streaming output truncated to the last 5000 lines.
Train Epoch: 4 [41408/60000 (69%)]	Loss: 0.186409
Train Epoch: 4 [41472/60000 (69%)]	Loss: 0.037771
Train Epoch: 4 [41536/60000 (69%)]	Loss: 0.162810
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.093270
Train Epoch: 4 [41664/60000 (69%)]	Loss: 0.138423
Train Epoch: 4 [41728/60000 (70%)]	Loss: 0.121513
Train Epoch: 4 [41792/60000 (70%)]	Loss: 0.076178
Train Epoch: 4 [41856/60000 (70%)]	Loss: 0.124076
Train Epoch: 4 [41920/60000 (70%)]	Loss: 0.116439
Train Epoch: 4 [41984/60000 (70%)]	Loss: 0.142701
Train Epoch: 4 [42048/60000 (70%)]	Loss: 0.023213
Train Epoch: 4 [42112/60000 (70%)]	Loss: 0.115163
Train Epoch: 4 [42176/60000 (70%)]	Loss: 0.107249
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.048453
Train Epoch: 4 [42304/60000 (70%)]	Loss: 0.040233
Train Epoch: 4 [42368/60000 (71%)]	Loss: 0.134970
Train Epoch: 4 [42432/60000 (71%)]	Loss: 0.029940
Train Epoch: 4 [42496/60000 (71%)]	Loss: 0.046528
Train Epoch: 4 [42560/60000 (71%)]	Loss: 0.061681